In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.layers import Dense, Dropout, SimpleRNN, LSTM
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import sys, os
sys.path.append(os.path.join(os.path.dirname('Demand'), '..', 'src'))
from Demand import Demand

In [2]:
path = '../data/demand_lower_48'

In [3]:
nat_dem = Demand()

In [4]:
nat_dem.load_and_clean_data(path)

In [5]:
df = nat_dem.dataframe

In [6]:
df.head()

,Megawatthours,Year,Month,Hour,Day_of_week,Day_of_month,Day_of_year
Time,,,,,,,
2015-07-01 02:00:00,335153,2015,7,2,2,1,182
2015-07-01 03:00:00,333837,2015,7,3,2,1,182
2015-07-01 04:00:00,398386,2015,7,4,2,1,182
2015-07-01 05:00:00,388954,2015,7,5,2,1,182
2015-07-01 06:00:00,392487,2015,7,6,2,1,182


In [7]:
df = df.drop(columns=['Day_of_week', 'Day_of_month', 'Day_of_year'])

In [8]:
df = df.reset_index()

In [9]:
df.head()

,Time,Megawatthours,Year,Month,Hour
0,2015-07-01 02:00:00,335153,2015,7,2
1,2015-07-01 03:00:00,333837,2015,7,3
2,2015-07-01 04:00:00,398386,2015,7,4
3,2015-07-01 05:00:00,388954,2015,7,5
4,2015-07-01 06:00:00,392487,2015,7,6


In [10]:
df['sin_day'] = [np.sin(x * (2*np.pi/24)) for x in df['Hour']]
df['cos_day'] = [np.cos(x * (2*np.pi/24)) for x in df['Hour']]

In [11]:
df.head(30)

,Time,Megawatthours,Year,Month,Hour,sin_day,cos_day
0,2015-07-01 02:00:00,335153,2015,7,2,5.000000e-01,8.660254e-01
1,2015-07-01 03:00:00,333837,2015,7,3,7.071068e-01,7.071068e-01
2,2015-07-01 04:00:00,398386,2015,7,4,8.660254e-01,5.000000e-01
3,2015-07-01 05:00:00,388954,2015,7,5,9.659258e-01,2.588190e-01
4,2015-07-01 06:00:00,392487,2015,7,6,1.000000e+00,6.123234e-17
5,2015-07-01 07:00:00,404647,2015,7,7,9.659258e-01,-2.588190e-01
6,2015-07-01 08:00:00,422227,2015,7,8,8.660254e-01,-5.000000e-01
7,2015-07-01 09:00:00,442131,2015,7,9,7.071068e-01,-7.071068e-01
8,2015-07-01 10:00:00,464371,2015,7,10,5.000000e-01,-8.660254e-01
9,2015-07-01 11:00:00,491512,2015,7,11,2.588190e-01,-9.659258e-01


In [12]:
df['Timestamp'] = [x.timestamp() for x in df['Time']]

In [13]:
df.head()

,Time,Megawatthours,Year,Month,Hour,sin_day,cos_day,Timestamp
0,2015-07-01 02:00:00,335153,2015,7,2,0.500000,8.660254e-01,1.435716e+09
1,2015-07-01 03:00:00,333837,2015,7,3,0.707107,7.071068e-01,1.435720e+09
2,2015-07-01 04:00:00,398386,2015,7,4,0.866025,5.000000e-01,1.435723e+09
3,2015-07-01 05:00:00,388954,2015,7,5,0.965926,2.588190e-01,1.435727e+09
4,2015-07-01 06:00:00,392487,2015,7,6,1.000000,6.123234e-17,1.435730e+09


In [14]:
s= 24*60*60
year = 365.25*s
df["sin_month"] = [np.sin((x) * (2 * np.pi / year)) for x in df["Timestamp"]]
df["cos_month"] = [np.cos((x) * (2 * np.pi / year)) for x in df["Timestamp"]]


In [15]:
df.head()

,Time,Megawatthours,Year,Month,Hour,sin_day,cos_day,Timestamp,sin_month,cos_month
0,2015-07-01 02:00:00,335153,2015,7,2,0.500000,8.660254e-01,1.435716e+09,0.030816,-0.999525
1,2015-07-01 03:00:00,333837,2015,7,3,0.707107,7.071068e-01,1.435720e+09,0.030100,-0.999547
2,2015-07-01 04:00:00,398386,2015,7,4,0.866025,5.000000e-01,1.435723e+09,0.029383,-0.999568
3,2015-07-01 05:00:00,388954,2015,7,5,0.965926,2.588190e-01,1.435727e+09,0.028667,-0.999589
4,2015-07-01 06:00:00,392487,2015,7,6,1.000000,6.123234e-17,1.435730e+09,0.027950,-0.999609


In [16]:
df.set_index('Time', inplace=True)

In [17]:
df = df[['Megawatthours', 'sin_day', 'cos_day', 'sin_month', 'cos_month']]

In [24]:
def create_X_Y(ts: np.array, lag=1, n_ahead=1, target_index=0) -> tuple:
    """
    A method to create X and Y matrix from a time series array for the training of 
    deep learning models 
    """
    # Extracting the number of features that are passed from the array 
    n_features = ts.shape[1]
    
    # Creating placeholder lists
    X, Y = [], []

    if len(ts) - lag <= 0:
        X.append(ts)
    else:
        for i in range(len(ts) - lag - n_ahead):
            Y.append(ts[(i + lag):(i + lag + n_ahead), target_index])
            X.append(ts[i:(i + lag)])

    X, Y = np.array(X), np.array(Y)

    # Reshaping the X array to an RNN input shape 
    X = np.reshape(X, (X.shape[0], lag, n_features))

    return X, Y
    

In [19]:
class NNMultistepModel():
    
    def __init__(
        self, 
        X, 
        Y, 
        n_outputs,
        n_lag,
        n_ft,
        n_layer,
        batch,
        epochs, 
        lr,
        Xval=None,
        Yval=None,
        mask_value=-999.0,
        min_delta=0.001,
        patience=5
    ):
        lstm_input = Input(shape=(n_lag, n_ft))

        # Series signal 
        lstm_layer = LSTM(n_layer, activation='relu')(lstm_input)

        x = Dense(n_outputs)(lstm_layer)
        
        self.model = Model(inputs=lstm_input, outputs=x)
        self.batch = batch 
        self.epochs = epochs
        self.n_layer=n_layer
        self.lr = lr 
        self.Xval = Xval
        self.Yval = Yval
        self.X = X
        self.Y = Y
        self.mask_value = mask_value
        self.min_delta = min_delta
        self.patience = patience

    def trainCallback(self):
        return EarlyStopping(monitor='loss', patience=self.patience, min_delta=self.min_delta)

    def train(self):
        # Getting the untrained model 
        empty_model = self.model
        
        # Initiating the optimizer
        optimizer = keras.optimizers.Adam(learning_rate=self.lr)

        # Compiling the model
        empty_model.compile(loss=losses.MeanAbsoluteError(), optimizer=optimizer)

        if (self.Xval is not None) & (self.Yval is not None):
            history = empty_model.fit(
                self.X, 
                self.Y, 
                epochs=self.epochs, 
                batch_size=self.batch, 
                validation_data=(self.Xval, self.Yval), 
                shuffle=False,
                callbacks=[self.trainCallback()]
            )
        else:
            history = empty_model.fit(
                self.X, 
                self.Y, 
                epochs=self.epochs, 
                batch_size=self.batch,
                shuffle=False,
                callbacks=[self.trainCallback()]
            )
        
        # Saving to original model attribute in the class
        self.model = empty_model
        
        # Returning the training history
        return history
    
    def predict(self, X):
        return self.model.predict(X)

In [20]:
# Number of lags (hours back) to use for models
lag = 48# Steps ahead to forecast 
n_ahead = 1# Share of obs in testing 
test_share = 0.1# Epochs for training
epochs = 20# Batch size 
batch_size = 512# Learning rate
lr = 0.001# Number of neurons in LSTM layer
n_layer = 10

In [21]:
features_final = ['Megawatthours', 'sin_day', 'cos_day', 'sin_month', 'cos_month']
X_features = features_final[1:]

In [29]:
ts = df[features_final]
nrows = ts.shape[0]# Spliting into train and test sets
train = ts[0:int(-nrows * test_share)]
test = ts[int(-nrows * test_share):]# Scaling the data 
train_mean = train.mean()
train_std = train.std()
train = (train - train_mean) / train_std
test = (test - train_mean) / train_std# Creating the final scaled frame 
ts_s = pd.concat([train, test])# Creating the X and Y for training
X, Y = create_X_Y(ts_s.values, lag=lag, n_ahead=n_ahead)
X = X[:, : ,1:]
n_ft = X.shape[2]

In [30]:
Xtrain, Ytrain = X[0:int(X.shape[0] * (1 - test_share))], Y[0:int(X.shape[0] * (1 - test_share))]

In [31]:
Xval, Yval = X[int(X.shape[0] * (1 - test_share)):], Y[int(X.shape[0] * (1 - test_share)):]

In [32]:
Xtrain.shape

(45117, 48, 4)

In [33]:
Ytrain.shape

(45117, 1)

In [34]:
Ytrain

array([[-0.53954258],
       [-0.81311575],
       [-0.95762332],
       ...,
       [ 2.8588372 ],
       [ 2.60401871],
       [ 2.2656258 ]])

In [35]:
Xval.shape

(5013, 48, 4)

In [36]:
Yval.shape

(5013, 1)

In [37]:
import keras
from keras import losses, Model, Input
from keras.callbacks import EarlyStopping

In [ ]:
# Initiating the model object
model = NNMultistepModel(
 X=Xtrain,
 Y=Ytrain,
 n_outputs=n_ahead,
 n_lag=lag,
 n_ft=n_ft,
 n_layer=n_layer,
 batch=batch_size,
 epochs=epochs, 
 lr=lr,
 Xval=Xval,
 Yval=Yval,
)# Training of the model 
history = model.train()

Epoch 1/20
89/89 [==============================] - 1s 14ms/step - loss: 0.8546 - val_loss: 0.5188
Epoch 2/20
89/89 [==============================] - 1s 11ms/step - loss: 0.7142 - val_loss: 0.4827
Epoch 3/20
89/89 [==============================] - 1s 11ms/step - loss: 0.5402 - val_loss: 0.4217
Epoch 4/20
89/89 [==============================] - 1s 11ms/step - loss: 0.4108 - val_loss: 0.3920
Epoch 5/20
89/89 [==============================] - 1s 11ms/step - loss: 0.3997 - val_loss: 0.3718
Epoch 6/20
89/89 [==============================] - 1s 11ms/step - loss: 0.3715 - val_loss: 0.3758
Epoch 7/20
89/89 [==============================] - 1s 11ms/step - loss: 0.3627 - val_loss: 0.3647
Epoch 8/20
61/89 [===================>..........] - ETA: 0s - loss: 0.3737

In [ ]:
# Comparing the forecasts with the actual values
yhat = [x[0] for x in model.predict(Xval)]
y = [y[0] for y in Yval]
# Creating the frame to store both predictions
days = df.index[-len(y):]
frame = pd.concat([
 pd.DataFrame({'day': days, 'Megawatthours': y, 'type': 'original'}),
 pd.DataFrame({'day': days, 'Megawatthours': yhat, 'type': 'forecast'})
])

In [ ]:
frame

In [ ]:
frame.columns

In [ ]:
train_std

In [ ]:
# Creating the unscaled values column
frame['MWH'] = [(x * train_std['Megawatthours']) + train_mean['Megawatthours'] for x in frame['Megawatthours']]

In [ ]:
frame

In [ ]:
len(y)

In [ ]:
len(yhat)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(df.index[-len(y):], y)
ax.plot(df.index[-len(y):], yhat, alpha=.5)
plt.show()

In [ ]:
df.index